In [2]:
!nvidia-smi

Wed Apr 17 11:56:26 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.147.05   Driver Version: 525.147.05   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 31%   52C    P0    39W / 170W |   1117MiB /  8192MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%%writefile reduce.cu
/* This program will performe a reduce vectorA (size N)
* with the + operation.
+---------+ 
|111111111| 
+---------+
     |
     N

vectorA   = all Ones
N = Sum of vectorA
*/
#include <iostream>
#include <sstream>
#include <stdlib.h>
#include "cuda_runtime.h"
#include "device_launch_parameters.h"

using namespace std;


// CUDA macro wrapper for checking errors
#define gpuErrCheck(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char* file, int line, bool abort = true)
{
    if (code != cudaSuccess)
    {
        std::cout << "GPUassert: " << cudaGetErrorString(code) << " " << file << " " << line << std::endl;
        if (abort)
        {
            exit(code);
        }
    }
}


// CPU reduce
void reduce(int* vectorA, int* sum, int size)
{
    sum[0] = 0;
    for (int i = 0; i < size; i++)
        sum[0] += vectorA[i];
}


// EXERCISE
// Read: https://developer.nvidia.com/blog/faster-parallel-reductions-kepler/
// Implement the reduce kernel based on the information
// of the Nvidia blog post.
// Implement both options, using no shared mem at all but global atomics
// and using shared mem for the seconds recution phase.
__global__ void cudaEvenFasterReduceAddition() {
    //ToDo
}


// Already optimized reduce kernel using shared memory.
__global__ void cudaReduceAddition(int* vectorA, int* sum)
{
    int globalIdx = 2 * blockDim.x * blockIdx.x + threadIdx.x;
    extern __shared__ int shmArray[];

    shmArray[threadIdx.x] = vectorA[globalIdx];
    shmArray[threadIdx.x + blockDim.x] = vectorA[globalIdx + blockDim.x];

    for (int stride = blockDim.x; stride; stride >>= 1) {
        if (threadIdx.x < stride) {
            shmArray[threadIdx.x] += shmArray[threadIdx.x + stride];
        }
        __syncthreads();
    }

    if (threadIdx.x == 0) {
        sum[blockIdx.x] = shmArray[0];
    }
}


// Compare result vectors
int compareResultVec(int* vectorCPU, int* vectorGPU, int size)
{
    int error = 0;
    for (int i = 0; i < size; i++)
    {
        error += abs(vectorCPU[i] - vectorGPU[i]);
    }
    if (error == 0)
    {
        cout << "No errors. All good!" << endl;
        return 0;
    }
    else
    {
        cout << "Accumulated error: " << error << endl;
        return -1;
    }
}


int main(void)
{
    // Define the size of the vector: 1048576 elements
    const int N = 1 << 20;
    const int NBR_BLOCK = 512;

    // Allocate and prepare input
    int* hostVectorA = new int[N];
    int hostSumCPU[1];
    int hostSumGPU[1];
    for (int i = 0; i < N; i++) {
        hostVectorA[i] = 1;
    }

    // Alloc N times size of int at address of deviceVector[A-C]
    int* deviceVectorA;
    int* deviceSum;
    gpuErrCheck(cudaMalloc(&deviceVectorA, N * sizeof(int)));
    gpuErrCheck(cudaMalloc(&deviceSum, NBR_BLOCK* sizeof(int)));

    // Copy data from host to device
    gpuErrCheck(cudaMemcpy(deviceVectorA, hostVectorA, N * sizeof(int), cudaMemcpyHostToDevice));

    // Run the vector kernel on the CPU
    reduce(hostVectorA, hostSumCPU, N);

    // Run kernel on all elements on the GPU
    cudaReduceAddition <<<NBR_BLOCK, 1024, 2 * 1024 * sizeof(int)>>> (deviceVectorA, deviceSum);
    gpuErrCheck(cudaPeekAtLastError());
    cudaReduceAddition <<<1, NBR_BLOCK / 2, NBR_BLOCK * sizeof(int) >> > (deviceSum, deviceSum);
    gpuErrCheck(cudaPeekAtLastError());

    // Copy the result stored in device_y back to host_y
    gpuErrCheck(cudaMemcpy(hostSumGPU, deviceSum, sizeof(int), cudaMemcpyDeviceToHost));

    // Check for errors
    auto isValid = compareResultVec(hostSumCPU, hostSumGPU, 1);

    // Free memory on device
    gpuErrCheck(cudaFree(deviceVectorA));
    gpuErrCheck(cudaFree(deviceSum));

    // Free memory on host
    delete[] hostVectorA;

    return isValid;
}


Writing reduce.cu


**Attention:** If you get a K80, you should compile it like this:
`!nvcc -arch=sm_37 -o reduce reduce.cu`

In [4]:
!nvcc -o reduce reduce.cu

In [5]:
!./reduce

No errors. All good!


**Profiling on old GPU (K80)**

*   `!nvprof --print-gpu-trace ./reduce`
*   `!nvprof --analysis-metrics -o reduce_out.nvprof ./reduce`
*   --> Use the Visual Profiler

**Profiling on newer GPUs**

*   `!nsys profile -f true -o reduce_out -t cuda ./reduce`
*   `!nsys stats --report gputrace reduce.qdrep`
*   `!nsys stats reduce.qdrep`
*   `!ncu -f -o reduce --set full ./reduce`
*   --> Nvidia Nsight Tools

Example:

In [7]:
!nvprof --print-gpu-trace ./reduce

==253== NVPROF is profiling process 253, command: ./reduce
No errors. All good!
==253== Profiling application: ./reduce
==253== Profiling result:
   Start  Duration            Grid Size      Block Size     Regs*    SSMem*    DSMem*      Size  Throughput  SrcMemType  DstMemType           Device   Context    Stream  Name
381.08ms  804.09us                    -               -         -         -         -  4.0000MB  4.8580GB/s    Pageable      Device     Tesla T4 (0)         1         7  [CUDA memcpy HtoD]
385.04ms  92.031us            (512 1 1)      (1024 1 1)        16        0B  8.0000KB         -           -           -           -     Tesla T4 (0)         1         7  cudaReduceAddition(int*, int*) [114]
385.14ms  6.3360us              (1 1 1)       (256 1 1)        16        0B  2.0000KB         -           -           -           -     Tesla T4 (0)         1         7  cudaReduceAddition(int*, int*) [116]
385.14ms  2.0160us                    -               -         -         - 

In [8]:
!nvprof --analysis-metrics -o reduce_out.nvprof ./reduce

======== Warning: Skipping profiling on device 0 since profiling is not supported on devices with compute capability 7.5 and higher.
                  Use NVIDIA Nsight Compute for GPU profiling and NVIDIA Nsight Systems for GPU tracing and CPU sampling.
                  Refer https://developer.nvidia.com/tools-overview for more details.

======== Warning: The option --aggregate-mode on has no effect. The --aggregate-mode <on|off> option applies to --events and --metrics options that follow it.
======== Warning: The option --aggregate-mode off has no effect. The --aggregate-mode <on|off> option applies to --events and --metrics options that follow it.
==264== NVPROF is profiling process 264, command: ./reduce
No errors. All good!
==264== Generated result file: /content/reduce_out.nvprof


Use the file *reduce_out.nvprof* with the visual profiler of Nvidia, which you have locally installed.

In [ ]:
!nsys profile -f true --stats=true -o reduce_out -t cuda ./reduce

In [ ]:
!nsys stats reduce_out.qdrep